In [17]:
# 必要なライブラリをインポートする。
from importlib import reload
from helper import run
import network

from block import GENESIS_BLOCK
from helper import calculate_new_bits
from network import (
    NetworkEnvelope,
    VersionMessage,
)

### 練習問題 1

Determine what this network message is:

`f9beb4d976657261636b000000000000000000005df6e0e2`

In [12]:
# 練習問題 1
from io import BytesIO
# ネットワークマジック
message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'
# convert to binary
stream = BytesIO(bytes.fromhex(message_hex))
# see what the bytes 4 through 16 aregi
command = NetworkEnvelope.parse(stream)
print(command.command)
print(command.payload)

b'verack'
b''


### 練習問題 2

Write the `parse` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_parse`

In [13]:
# 練習問題 2

reload(network)
run(network.NetworkEnvelopeTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### 練習問題 3

Write the `serialize` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_serialize`

In [14]:
# 練習問題 3

reload(network)
run(network.NetworkEnvelopeTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### 練習問題 4

Write the `serialize` method for `VersionMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:VersionMessageTest:test_serialize`

In [22]:
# 練習問題 4

reload(network)
run(network.VersionMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### 練習問題 5

Write the `handshake` method for `SimpleNode`

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:SimpleNodeTest:test_handshake`

In [24]:
# 練習問題 5

reload(network)
run(network.SimpleNodeTest("test_handshake"))

.
----------------------------------------------------------------------
Ran 1 test in 0.324s

OK


### 練習問題 6

Write the `serialize` method for `GetHeadersMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:GetHeadersMessageTest:test_serialize`

In [25]:
# 練習問題 6

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [26]:
# ハンドシェイクした後ブロックヘッダーを要求するソースコード
from io import BytesIO
from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import Block, GENESIS_BLOCK, LOWEST_BITS
from helper import calculate_new_bits
# ジェネシスブロックのデータをパースする。
previous = Block.parse(BytesIO(GENESIS_BLOCK))
first_epoch_timestamp = previous.timestamp
expected_bits = LOWEST_BITS
count = 1
# 対象ノード
node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
# ハンドシェイクを確立させる。
node.handshake()
for _ in range(19):
    # ブロックヘッダーの情報を取得する。
    getheaders = GetHeadersMessage(start_block=previous.hash())
    # ブロックヘッダーを送信する。
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for header in headers.blocks:
        if not header.check_pow(): # PoWが有効であることをチェックする。
            raise RuntimeError('bad PoW at block {}'.format(count))
        if header.prev_block != previous.hash(): # 前のブロックの次であることをチェックする。
            raise RuntimeError('discontinuous block at {}'.format(count))
        if count % 2016 == 0:
            # 2016ブロックごとにタイムスタンプの差分を計算する。
            time_diff = previous.timestamp - first_epoch_timestamp
            # 難易度を調整する。
            expected_bits = calculate_new_bits(previous.bits, time_diff)
            print(expected_bits.hex())
            first_epoch_timestamp = header.timestamp
        if header.bits != expected_bits:
            raise RuntimeError('bad bits at block {}'.format(count))
        previous = header
        count += 1

ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
6ad8001d
28c4001d
71be001d
